In [1]:
import urllib3
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from io import StringIO
import json
from headers import *
import controllers

import importlib
importlib.reload(controllers)
controller = controllers.FuturesDataController()

In [2]:
df = controller.crawler_twse_total_return("2024/09/10")
df

,date,tri_close
0,2024-09-02,48960.74
1,2024-09-03,48647.56
2,2024-09-04,46447.84
3,2024-09-05,46663.02
4,2024-09-06,47208.33
5,2024-09-09,46568.93
6,2024-09-10,46391.97
7,2024-09-11,46319.12
8,2024-09-12,47767.18
9,2024-09-13,48002.35


In [5]:
df2 = controller.crawler_twse_data("2024/09/010")
df2

爬取日期: 20240909


,date,open,high,low,close
0,2024-09-02,22341.9,22439.31,22178.04,22235.1
1,2024-09-03,22240.11,22303.5,22092.21,22092.21
2,2024-09-04,21574.65,21574.65,20922.17,21092.75
3,2024-09-05,21271.09,21488.81,21150.99,21187.71
4,2024-09-06,21255.45,21461.71,21125.14,21435.19
5,2024-09-09,21127.81,21163.91,20922.51,21144.44
6,2024-09-10,21252.57,21324.43,20977.0,21064.08


In [2]:
url = "https://www.twse.com.tw/fund/TWT38U"
date = "20240909"
params = {
    'date':f'{date}'
}

response = requests.get(url,data=params,headers=TWSE_DOWN)

In [ ]:
data = json.loads(response.text)
data

In [ ]:
columns = data['fields']
rows = data['data']
df = pd.DataFrame(rows,columns=columns).iloc[:,:6]
df

In [ ]:
df['鉅額成交'] = 0
df.loc[df.iloc[:,0] == '*','鉅額成交'] = 1 
df

In [ ]:
df = df.replace(',','',regex=True)
df['date'] = date
df['date'] = pd.to_datetime(df['date'])
df = df[['date'] + [col for col in df.columns if col!='date']]
df

In [ ]:
df = controller.crawler_twse_trade_amount("2024/09/09")
df

In [ ]:
url = "https://www.twse.com.tw/rwd/fund/BFI82U"
date = "20240909"
params = {
    'response':'json',
    'dayDate':f'{date}',
    'type' : 'day'

}
response = requests.get(url,data=params,headers=TWSE_DOWN)
response.raise_for_status()

response.text

In [5]:
data = json.loads(response.text)

columns = data['fields']
rows = data['data']

df = pd.DataFrame(rows,columns = columns)

In [ ]:
df.iloc[:,1:] = df.iloc[:,1:].apply(lambda x:x.str.replace(',','').astype('int64'),axis=1)
df

In [ ]:
df['date'] = "2024/09/09"
df['date'] = pd.to_datetime(df['date'])
df

In [ ]:
df =df[df['單位名稱'] != "合計"]
df

In [ ]:
df = df[['date'] + [col for col in df.columns if col != 'date']]
df